--- Day 3: Binary Diagnostic ---

The submarine has been making some odd creaking noises, so you ask it to produce a diagnostic report just in case.

The diagnostic report (your puzzle input) consists of a list of binary numbers which, when decoded properly, can tell you many useful things about the conditions of the submarine. The first parameter to check is the **power consumption**.

You need to use the binary numbers in the diagnostic report to generate two new binary numbers (called the **gamma rate** and the **epsilon rate**). The power consumption can then be found by multiplying the gamma rate by the epsilon rate.

Each bit in the gamma rate can be determined by finding the **most common bit in the corresponding position** of all numbers in the diagnostic report. For example, given the following diagnostic report:

```
00100
11110
10110
10111
10101
01111
00111
11100
10000
11001
00010
01010
```

Considering only the first bit of each number, there are five `0` bits and seven `1` bits. Since the most common bit is `1`, the first bit of the gamma rate is `1`.

The most common second bit of the numbers in the diagnostic report is `0`, so the second bit of the gamma rate is `0`.

The most common value of the third, fourth, and fifth bits are `1`, `1`, and `0`, respectively, and so the final three bits of the gamma rate are `110`.

So, the gamma rate is the binary number `10110`, or `22` in decimal.

The epsilon rate is calculated in a similar way; rather than use the most common bit, the least common bit from each position is used. So, the epsilon rate is `01001`, or `9` in decimal. Multiplying the gamma rate (`22`) by the epsilon rate (`9`) produces the power consumption, `198`.

Use the binary numbers in your diagnostic report to calculate the gamma rate and epsilon rate, then multiply them together. **What is the power consumption of the submarine**? (Be sure to represent your answer in decimal, not binary.)

## Solution

I'm assuming that all the bit lengths are the same, but it's good to check.  So I'll write a quick function to add all the bit lengths to a set to remove any duplicate lengths.  If the set returned has more than one length then that might change the method I need to use to determine the most common bits.

In [1]:
def get_bit_length(filename: str) -> set:
    bit_lengths = set()
    with open(filename, 'r') as f:
        for line in f:
            bit_lengths.add(len(line.strip()))
    return bit_lengths

sample_lengths = get_bit_length('sample.txt')
input_lengths = get_bit_length('input.txt')
print(f"Sample bit lengths: {sample_lengths}\nInput Lengths: {input_lengths}\n")

Sample bit lengths: {5}
Input Lengths: {12}



Great!  The bit lengths are all consistent per file, so that will make calculating the most common bits per file easier.  However, since the sample has a different bit length than the input, I'll want to account for variability per file.  In order to take advantage of the file object iterator, I can use a matrix to store a counter for each bit position.

In [2]:
from typing import List


def build_matrix(length: int) -> List[List[int]]:
    matrix = [[0, 0] for idx in range(0, length)]
    return matrix

sample_matrix = build_matrix(5)
print(sample_matrix)

[[0, 0], [0, 0], [0, 0], [0, 0], [0, 0]]


I'll probably also want a function that can take the matrix and the line of input and increment all the counters.

In [3]:
def count_bits(matrix: List[List[int]], line: str) -> List[List[int]]:
    for i in range(0, len(line)):
        if line[i] == '0':
            matrix[i][0] += 1
        else:
            matrix[i][1] += 1
    return matrix

sample_counts = count_bits(sample_matrix, '00100')
print(sample_counts)

[[1, 0], [1, 0], [0, 1], [1, 0], [1, 0]]


Next I'll need a function to build the string with the most common bits by getting the position of the maximum of each bit counter.

In [4]:
def calc_gamma(matrix: List[List[int]]) -> str:
    gamma = ''
    for bit_counts in matrix:
        gamma += str(bit_counts.index(max(bit_counts)))
    return int(str(gamma), 2)

sample_gamma = calc_gamma(sample_matrix)
print(sample_gamma)

4


Now I think we can string it all together.  I'll need to build the appropriate sized matrix based on the length of the first line and then count the bits for each line.  Finally I'll need to calculate the gamma.  I'll also need the bit length to calculate epsilon.

In [5]:
def get_gamma(filename: str) -> int:
    with open(filename, 'r') as f:
        head = next(f).strip()
        matrix = build_matrix(len(head))
        matrix = count_bits(matrix, head)
        for line in f:
            matrix = count_bits(matrix, line.strip())
    return (calc_gamma(matrix), len(head))


sample_result = get_gamma('sample.txt')
print(sample_result)

(22, 5)


Now that I have gamma, I should be able to calculate epsilon from that.  There's probably a fancy way to do that using XOR or something, but I'm not familiar with it.  So this iteration just gets a function that converts the decimal into binary, reverses the bits, and then converts back to decimal.  Ugly, but effective.  I need the bit length to ensure that the if there are leading zeros in the binary representation of gamma that they are recorded properly.  If they are stripped by the formatting, then the conversion will produce an incorrect value due to the reduce number of bits to convert.

In [6]:
def get_epsilon(gamma: int, bits: int) -> int:
    gamma_bit_string = format(gamma, F"0{bits}b")
    epsilon_bit_string = ''
    for bit in gamma_bit_string:
        if bit == '0':
            epsilon_bit_string += '1'
        else:
            epsilon_bit_string += '0'
    return int(epsilon_bit_string, 2)

print(get_epsilon(*sample_result))

9


I think we have all the pieces.  We can test using the sample and then get the puzzle answer if everything looks good.

In [7]:
sample_gamma = get_gamma('sample.txt')
sample_epsilon = get_epsilon(*sample_gamma)
sample_answer = sample_gamma[0] * sample_epsilon
print(f"The sample gamma, {sample_gamma[0]}, multiplied by the sample epsilon, {sample_epsilon}, equals {sample_answer}")
puzzle_gamma = get_gamma('input.txt')
puzzle_epsilon = get_epsilon(*puzzle_gamma)
puzzle_answer = puzzle_gamma[0] * puzzle_epsilon
print(f"The puzzle gamma, {puzzle_gamma[0]}, multiplied by the puzzle epsilon, {puzzle_epsilon}, equals {puzzle_answer}")


The sample gamma, 22, multiplied by the sample epsilon, 9, equals 198
The puzzle gamma, 1565, multiplied by the puzzle epsilon, 2530, equals 3959450
